In [ ]:
pip install paddleocr groq

In [ ]:
import os
import logging
from flask import Flask, request, render_template, jsonify
from groq import Groq
from paddleocr import PaddleOCR
import csv

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

app = Flask(__name__)

# Initialize OCR and Groq
try:
    ocr = PaddleOCR(use_angle_cls=True, lang='en')
    logging.info("PaddleOCR initialized successfully")
except Exception as e:
    logging.error(f"Failed to initialize PaddleOCR: {str(e)}")
    raise

try:
    client = Groq(api_key="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")
    logging.info("Groq client initialized successfully")
except Exception as e:
    logging.error(f"Failed to initialize Groq client: {str(e)}")
    raise

# Global variables
image_path = None
csv_path = 'data.csv'
log_csv_path = 'refined_text_log.csv'

# Original logic functions (unchanged)
def log_refined_text(refined_text):
    """
    Logs the refined OCR-extracted text into a CSV file.
    """
    try:
        with open(log_csv_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([refined_text])
        logging.debug(f"Logged refined text to {log_csv_path}")
    except Exception as e:
        logging.error(f"Error logging refined text: {str(e)}")
        raise

def extract_and_clean_text(image_path):
    """
    Extracts text from an image using OCR and cleans it.
    """
    try:
        logging.debug(f"Processing image with OCR: {image_path}")
        results = ocr.ocr(image_path, cls=True)
        if not results or not results[0]:
            logging.warning(f"No text detected in image: {image_path}")
            raise ValueError("No readable text detected in the image")
        logging.debug(f"OCR results: {results}")
        cleaned_text = []
        for line in results[0]:
            text = line[1][0]
            if len(text) >= 3 and text.isprintable():
                cleaned_text.append(text)
        logging.debug(f"Cleaned text: {cleaned_text}")
        return cleaned_text
    except Exception as e:
        logging.error(f"Error in extract_and_clean_text: {str(e)}")
        raise

def process_image_and_csv(image_path, csv_path):
    """
    Processes a nutritional image, sending data to the Groq API for analysis.
    """
    if not image_path:
        logging.error("No image path provided")
        return "Error: No image path provided!", "", ""
    try:
        cleaned_text = extract_and_clean_text(image_path)
        extracted_text = ", ".join(cleaned_text)
        logging.debug(f"Extracted text: {extracted_text}")
        prompt = f"""
        I am using OCR to extract the Nutritional information from the Food pack labels. 
        I need you to refine the text: {extracted_text}. Just return the nutritional facts and Ingredients. 
        Based on the ingredients, what is the Food name? No other words.
        """
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "system", "content": "You are a professional medical advisor."},
                      {"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=300,
            top_p=1,
            stream=False
        )
        logging.debug(f"Groq response: {response}")
        refined_text = response.choices[0].message.content
        log_refined_text(refined_text)
        return refined_text, "", ""
    except Exception as e:
        logging.error(f"Error in process_image_and_csv: {str(e)}")
        return f"Error occurred: {str(e)}", "", ""

# Flask setup
UPLOAD_FOLDER = 'static/uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload_nutritional', methods=['POST'])
def upload_nutritional():
    global image_path
    if 'file' not in request.files:
        logging.error("No file part in request")
        return jsonify({"error": "No file uploaded"}), 400
    file = request.files['file']
    if file.filename == '':
        logging.error("No file selected")
        return jsonify({"error": "No file selected"}), 400
    try:
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        logging.debug(f"Saving file to: {filepath}")
        file.save(filepath)
        if not os.path.exists(filepath):
            logging.error("File save failed")
            return jsonify({"error": "Failed to save the uploaded file"}), 500
        image_path = filepath
        refined_text, _, _ = process_image_and_csv(image_path, csv_path)
        if refined_text.startswith("Error occurred:"):
            logging.error(f"Processing failed: {refined_text}")
            return jsonify({"error": refined_text}), 500
        logging.info("Nutritional image processed successfully")
        return jsonify({"refined_text": refined_text, "image_url": f"/{filepath}"})
    except Exception as e:
        logging.error(f"Upload nutritional error: {str(e)}")
        return jsonify({"error": f"Server error: {str(e)}"}), 500

@app.route('/upload_medical', methods=['POST'])
def upload_medical():
    if 'file' not in request.files:
        logging.error("No file part in request")
        return jsonify({"error": "No file uploaded"}), 400
    file = request.files['file']
    if file.filename == '':
        logging.error("No file selected")
        return jsonify({"error": "No file selected"}), 400
    try:
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        logging.debug(f"Saving file to: {filepath}")
        file.save(filepath)
        if not os.path.exists(filepath):
            logging.error("File save failed")
            return jsonify({"error": "Failed to save the uploaded file"}), 500
        extracted_text = extract_and_clean_text(filepath)
        medical_prompt = f"""
        I am using OCR to extract the text from a medical report. 
        Refine the text: {extracted_text}, remove any noise, and provide a clear summary of the medical findings. 
        Just return the important medical details and diagnosis if available, no extra words.
        """
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "system", "content": "You are a professional medical advisor."},
                      {"role": "user", "content": medical_prompt}],
            temperature=0.7,
            max_tokens=300,
            top_p=1,
            stream=False
        )
        logging.info("Medical report processed successfully")
        return jsonify({"refined_text": response.choices[0].message.content})
    except Exception as e:
        logging.error(f"Upload medical error: {str(e)}")
        return jsonify({"error": f"Server error: {str(e)}"}), 500

@app.route('/evaluate_combined', methods=['POST'])
def evaluate_combined():
    data = request.json
    nutritional_text = data.get('nutritional_text', '').strip()
    medical_text = data.get('medical_text', '').strip()
    selected_model = data.get('model', 'llama-3.3-70b-versatile')
    selected_language = data.get('language', 'English')

    if not nutritional_text:
        logging.error("No nutritional text provided")
        return jsonify({"error": "Please analyze nutritional data first"}), 400
    if not medical_text:
        logging.error("No medical text provided")
        return jsonify({"error": "Please process medical report first"}), 400

    try:
        next_prompt = f"""
        Dear User,

        Based on the extracted text from your food pack labels: {nutritional_text},
        and the details from your medical report: {medical_text},
        please evaluate the ingredients for safety.
        Provide a short recommendation on whether the food is safe to consume,
        including the safe quantity for intake if applicable.
        If the food is not recommended, briefly explain why it should be avoided.

        Please provide the response in the following format:

        1. First, a short and clear recommendation in **English**.
        2. After that, a short and clear recommendation in **{selected_language}** that corresponds to the English response.
        """
        final_response = client.chat.completions.create(
            model=selected_model,
            messages=[{"role": "system", "content": "You are a professional medical advisor."},
                      {"role": "user", "content": next_prompt}],
            temperature=0.7,
            max_tokens=400,
            top_p=1,
            stream=False
        )
        logging.info("Combined evaluation completed successfully")
        return jsonify({"result": final_response.choices[0].message.content})
    except Exception as e:
        logging.error(f"Evaluate combined error: {str(e)}")
        return jsonify({"error": f"Server error: {str(e)}"}), 500

if __name__ == "__main__":
    try:
        os.makedirs(UPLOAD_FOLDER, exist_ok=True)
        logging.info(f"Upload folder created/verified: {UPLOAD_FOLDER}")
        app.run(debug=True)
    except Exception as e:
        logging.error(f"Startup error: {str(e)}")

[INFO   ] [Logger      ] Record log in C:\Users\admin\.kivy\logs\kivy_25-02-28_4.txt
[INFO   ] [deps        ] Successfully imported "kivy_deps.angle" 0.4.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.glew" 0.3.1
[INFO   ] [deps        ] Successfully imported "kivy_deps.sdl2" 0.8.0
[INFO   ] [Kivy        ] v2.3.1
[INFO   ] [Kivy        ] Installed at "c:\Users\admin\Desktop\New folder\test\lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "c:\Users\admin\Desktop\New folder\test\Scripts\python.exe"
[INFO   ] [Logger      ] Purge log fired. Processing...
[INFO   ] [Logger      ] Purge finished!
[INFO   ] [Factory     ] 195 symbols loaded


[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil (img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sdl2
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succeeded
[INFO   ] [GL          ] Backend used <glew>
[INFO   ] [GL          ] OpenGL version <b'4.6.0 NVIDIA 566.07'>
[INFO   ] [GL          ] OpenGL vendor <b'NVIDIA Corporation'>
[INFO   ] [GL          ] OpenGL renderer <b'NVIDIA GeForce RTX 3060/PCIe/SSE2'>
[INFO   ] [GL          ] OpenGL parsed version: 4, 6
[INFO   ] [GL          ] Shading version <b'4.60 NVIDIA'>
[INFO   ] [GL          ] Texture max size <32768>
[INFO   ] [GL          ] Texture max units <32>
[INFO   ] [Window      ] auto add sdl2 input provider
[INFO   ] [Window      ] virtual keyboard not allowed, single mode, not docked


[2025/02/28 15:15:47] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\admin/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\admin/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

OSError: source code not available

In [1]:
pip install Flask

     ---------------------------------------- 0.0/103.0 kB ? eta -:--:--
     --- ------------------------------------ 10.2/103.0 kB ? eta -:--:--
     ------- ----------------------------- 20.5/103.0 kB 162.5 kB/s eta 0:00:01
     ------------------ ------------------ 51.2/103.0 kB 327.7 kB/s eta 0:00:01
     ------------------------------------ 103.0/103.0 kB 539.2 kB/s eta 0:00:00
     ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
     ---------------------------------------- 98.2/98.2 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/224.5 kB ? eta -:--:--
     ------- ------------------------------- 41.0/224.5 kB 1.9 MB/s eta 0:00:01
     ------- ------------------------------- 41.0/224.5 kB 1.9 MB/s eta 0:00:01
     ------------------ ----------------- 112.6/224.5 kB 819.2 kB/s eta 0:00:01
     -------------------------------------- 224.5/224.5 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/134.9 kB ? eta -:-


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install 